In [1]:
import numpy as np
import healpy as hp
import astropy.coordinates as coord
import astropy.units as u
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
#plt.style.use('SVA1StyleSheet.mplstyle')
%matplotlib inline

/usr/lib/python2.7/site-packages/healpy/pixelfunc.py:107: UserWarning: Warning: cannot import _healpy_pixel_lib module
  warnings.warn("Warning: cannot import _healpy_pixel_lib module")


ImportError: libcfitsio.so.7: cannot open shared object file: No such file or directory

In [ ]:
def make_healpix_map(ra, dec, quantity, nside, mask=None, weight=None, fill_UNSEEN=False, return_extra=False):
    """
    Creates healpix maps of quantity observed at ra, dec (in degrees) by taking
    the average of quantity in each pixel.
    Parameters
    ----------
    ra : array
        Right ascension.
    dec : array
        Declination.
    quantity : array
        `quantity` can be 2D, in which case several maps are created.
    nside : int
        `nside` parameter for healpix.
    mask : array
        If None, the mask is created and has value 1 in pixels that contain at
        least one object, 0 elsewhere.
    weight : type
        Weights of objects (the default is None, in which case all objects have
        weight 1). Must be the same size as `quantity`.
    fill_UNSEEN : boolean
        If `fill_UNSEEN` is True, pixels outside the mask are filled with
        hp.UNSEEN, 0 otherwise (the default is False).
    return_extra : boolean
        If True, a dictionnary is returned that contains count statistics and
        the masked `ipix` array to allow for statistics on the quantities to be
        computed.
    Returns
    -------
    List of outmaps, the count map and the mask map.
    """

    if quantity is not None:
        quantity = np.atleast_2d(quantity)

        if weight is not None:
            assert quantity.shape==weight.shape, "[make_healpix_map] quantity and weight must have the same shape"
            assert np.all(weight > 0.), "[make_healpix_map] weight is not strictly positive"
            weight = np.atleast_2d(weight)
        else:
            weight = np.ones_like(quantity)

        assert quantity.shape[1] == weight.shape[1], "[make_healpix_map] quantity/weight arrays don't have the same length"

        assert len(ra) == len(dec), "[make_healpix_map] ra/dec arrays don't have the same length"

    npix = hp.nside2npix(nside)

    if mask is not None:
        assert len(mask)==npix, "[make_healpix_map] mask array does not have the right length"

    # Value to fill outside the mask
    x = hp.UNSEEN if fill_UNSEEN else 0.0

    count = np.zeros(npix, dtype=float)
    outmaps = []

    # Getting pixels for each object
    ipix = hp.ang2pix(nside, (90.0-dec)/180.0*np.pi, ra/180.0*np.pi)

    # Counting objects in pixels
    np.add.at(count, ipix, 1.)

    # Creating the mask if it does not exist
    if mask is None:
        bool_mask = (count > 0)
    else:
        bool_mask = mask.astype(bool)

    # Masking the count in the masked area
    count[np.logical_not(bool_mask)] = x
    if mask is None:
        assert np.all(count[bool_mask] > 0), "[make_healpix_map] count[bool_mask] is not positive on the provided mask !"

    # Create the maps
    if quantity is not None:
        for i in range(quantity.shape[0]):
            sum_w = np.zeros(npix, dtype=float)
            np.add.at(sum_w, ipix, weight[i,:])

            outmap = np.zeros(npix, dtype=float)
            np.add.at(outmap, ipix, quantity[i,:]*weight[i,:])
            outmap[bool_mask] /= sum_w[bool_mask]
            outmap[np.logical_not(bool_mask)] = x

            outmaps.append(outmap)

    if mask is None:
        returned_mask = bool_mask.astype(float)
    else:
        returned_mask = mask

    if return_extra:
        extra = {}
        extra['count_tot_in_mask'] = np.sum(count[bool_mask])
        extra['count_per_pixel_in_mask'] = extra['count_tot_in_mask'] * 1. / np.sum(bool_mask.astype(int))
        extra['count_per_steradian_in_mask'] = extra['count_per_pixel_in_mask'] / hp.nside2pixarea(nside, degrees=False)
        extra['count_per_sqdegree_in_mask'] = extra['count_per_pixel_in_mask'] / hp.nside2pixarea(nside, degrees=True)
        extra['count_per_sqarcmin_in_mask'] = extra['count_per_sqdegree_in_mask'] / 60.**2
        extra['ipix_masked'] = np.ma.array(ipix, mask=bool_mask[ipix])

        return outmaps, count, returned_mask, extra
    else:
        return outmaps, count, returned_mask

In [ ]:
def thetaphi2radec(theta,phi):
    """
    Converts heta and phi (Healpix conventions, in radians) to ra and dec (in degrees)
    """
    ra = np.rad2deg(phi)
    dec = 90. - np.rad2deg(theta)
    return ra, dec
def maskmap(hpmap, binarymask, fill_UNSEEN=False):
    """
    Mask a map in place.
    Parameters
    ----------
    hpmap : array
        Input map.
    binarymask : array
        Mask.
    fill_UNSEEN : bool
        If True, fills pixels outside the mask with hp.UNSEEN, else with 0.0 (the default is False).
    Returns
    -------
    None
    """
    x = hp.UNSEEN if fill_UNSEEN else 0.0
    hpmap[np.logical_not(binarymask.astype(bool))] = x

def plot_skymapper(obs, mask, projection=None, title=None,filename=None, fsize=18, vmax=None, 
                   cmap=None, cb_label=None, nside_out=True, contrast=None):
    """
    Plot a healpix masked map using skymapper.
    Parameters
    ----------
    obs : array
        Healpix map.
    mask : array
        Mask map.
    projection : type
        - 'DESY3', in which case it uses precomputed best projection for the Y3 footprint,
        - a predefined skymapper projection objectself,
        - or None, if which case the projection is infered from the mask.
    title :string
        If not None, the name of the title of the figure (the default is None).
    filename : string
        If not None, the name of the file to save the figure (the default is None).
    vmax : type
        - a float, in which case the color scale goes from -vmax to +vmax
        - 'best', in which case skymapper uses the 10-90 percentiles
        -'lognorm', use log scale for the y axis
        - None, in which case the min/max values of `obs` are used.
    cmap : type
        Color map (the default is None).
    cb_label : string
        Label of the color bar (the default is None).
    nside_out : bool or int
        Whether to degrade obs to make if faster or the nside to use (the default
        is True, in which case nside=256 is used).
    contrast : type
        - 'neg', If True will only select from the obs healpix map only negatives values. Other values are
        set to zero.
        - 'abs', all the values are set to positive.
        - None, only plot to positive whe using log scale, otherwise all.
    Returns
    -------
    """
    
    import skymapper as skm
    import matplotlib.pyplot as plt
    from mpl_toolkits.axes_grid1 import make_axes_locatable

    assert obs.shape == mask.shape, "[plot_hp_skymapper] `obs` and `mask` don't have the same shape."

    fig = plt.figure(figsize=(10,6))
    ax = fig.add_subplot(111)

    nside_in = hp.npix2nside(len(mask))
    theta, phi = hp.pix2ang(nside_in, np.arange(len(mask))[mask.astype(bool)])
    ra, dec = thetaphi2radec(theta, phi)

    if projection == 'DESY3':
        # define the best Albers projection for the footprint
        # minimizing the variation in distortion
        # crit = skm.meanDistortion
        # proj = skm.Albers.optimize(a['ra'], a['dec'], crit=crit)
        proj = skm.Albers( 28.51234891, -43.90175288, -55.63596295, -32.39570739)
    else :
        if projection is None:
            crit = skm.meanDistortion
            proj = skm.Albers.optimize(ra, dec, crit=crit)

    # construct map: will hold figure and projection
    # the outline of the sphere can be styled with kwargs for matplotlib Polygon
    map = skm.Map(proj, ax=ax)
    
    map.footprint("DES", zorder=20, edgecolor='#2222B2', facecolor='None', lw=1)

    # add graticules, separated by 15 deg
    # the lines can be styled with kwargs for matplotlib Line2D
    # additional arguments for formatting the graticule labels
    #sep = 15
    #map.grid(sep)
    map.grid(15)
    map.focus(ra, dec)

    obs_plot = np.copy(obs)
    if contrast == 'neg':
        obs_plot[obs_plot>0]=0
        obs_plot[obs_plot<0]=-obs_plot[obs_plot<0]
    if contrast == 'abs':
        obs_plot[obs_plot<0]=-obs_plot[obs_plot<0]
        
    maskmap(obs_plot, mask, fill_UNSEEN=False)

    if nside_out:
        if type(nside_out) is int:
            obs_plot = hp.ud_grade(obs_plot, nside_out=nside_out)
        else:
            obs_plot = hp.ud_grade(obs_plot, nside_out=256)

    if type(vmax) is float:
        mappable = map.healpix(obs_plot, cmap=cmap, vmin=-vmax, vmax=vmax)
    if vmax == 'best' : # skymapper uses 10-90 percentiles
        mappable = map.healpix(obs_plot, cmap=cmap)
    if vmax == 'lognorm':
        mappable = map.healpix(obs_plot, cmap=cmap,norm=LogNorm( vmin=np.nanmin(obs_plot), 
                                                             vmax=np.nanmax(obs_plot))) 
    if vmax is None :
        mappable = map.healpix(obs_plot, cmap=cmap, vmin=np.min(obs_plot), vmax=np.max(obs_plot))
    
    #ticks=
    #map.colorbar(mappable, cb_label=cb_label)
    map.colorbar(mappable)
    
    
    if title is not None:
        plt.title(title,fontsize=fsize)

    plt.tight_layout()
    if filename is not None:
        plt.savefig(filename, dpi=300)

In [ ]:
def plotRaDec2D(infilename, field, nside=64,title='', fsize=18, outfilename=None,
                cmap_name='gnuplot',vmax=None, contrast=None, smooth=False, fwhm=0.01, savefig=False): 
    from astropy.coordinates import SkyCoord, Angle
    from astropy import units
    import skymapper as skm
    import fitsio

    data = fitsio.read(infilename)
    data = data.astype(data.dtype.newbyteorder('='))
    ignore =  (data[field] ==-999.) |  (data[field] == None) | ( np.isnan(data[field])) 
    data = data[~ignore]

    coords = SkyCoord(ra=data['telra'], dec=data['teldec'], unit='degree')
    ra = coords.ra.wrap_at(180 * units.deg).degree
    dec = coords.dec.degree
    rho2p = data[field]
    
    if not savefig: 
        outfilename = None

    outmaps, count, mask = make_healpix_map(ra, dec, rho2p,nside)
    map_out=outmaps[0]
    if smooth: map_out= hp.sphtfunc.smoothing(map_out, fwhm = fwhm, iter = 1)
    plot_skymapper(map_out, mask, projection='DESY3',title=title, fsize=fsize ,filename=outfilename,
                   vmax=vmax, cmap=cmap_name, nside_out=nside, contrast=contrast)
    
 

In [ ]:
filenames=['rho2pbyzones.fits', 'rho2pbyexposure.fits']
#cmaps=['gnuplot','seismic','Greys','tab20','tab20b','tab20c', 'ocean']
cmaps=['viridis', 'plasma', 'inferno', 'magma','Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
       'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu', 'GnBu', 'PuBu', 'YlGnBu','PuBuGn', 'BuGn', 'YlGn',
       'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink','spring', 'summer', 'autumn', 'winter', 
       'cool', 'Wistia', 'hot', 'afmhot', 'gist_heat', 'copper', 'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 
       'RdBu', 'RdYlBu', 'RdYlGn','Spectral', 'coolwarm', 'bwr', 'seismic', 'Pastel1', 'Pastel2', 'Paired', 
       'Accent', 'Dark2', 'Set1', 'Set2', 'Set3','tab10', 'tab20', 'tab20b', 'tab20c', 'flag', 'prism',
       'ocean', 'gist_earth', 'terrain', 'gist_stern','gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg', 
       'hsv', 'gist_rainbow', 'rainbow', 'jet', 'nipy_spectral', 'gist_ncar']
vmax=[None,'lognorm', 'best']
contrast=[None, 'neg', 'abs']
columns =  ['mrho2p', 'airmass', 'dimmseeing', 'dT', 'fwhm', 'humidity', 'msurtemp', 'outtemp', 'sat',  
            'sigsky', 'sky', 'teldec', 'telha', 'telra', 'tiling', 'mtotalstars', 'musestars', 'winddir',
            'windspd', 'mean_obs_e1', 'mean_obs_e2', 'mean_obs_e', 'mean_obs_epw2', 'mean_piff_e1',
            'mean_piff_e2', 'mean_piff_e', 'mean_piff_epw2', 'mean_de1', 'mean_de2', 'mean_de','mean_depw2']
units =  ['', '', '[arcsec]', '[Celcius]', '', '[%]', '[Celsius]', '[Celsius]', '', '', '', '[deg]', '[deg]',
          '[deg]', '', '', '', '[deg]', '[m/s]', '', '', '', '', '', '', '', '', '', '', '', '']
xtitles= np.core.defchararray.add(np.array(columns),np.array(units))

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

wnside = widgets.SelectionSlider(options=[2,4,8,16,32,64,128,256,512,1024],value=32,disabled=False,
                                 continuous_update=False,readout=True)
wvmax= widgets.FloatSlider(value=1.0e-10,min=1.0e-10,max=1.0e-8,step=1.0e-9,disabled=False,
                                 continuous_update=False,readout=True)
wfwhm = widgets.SelectionSlider(options=[0.0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.08,0.1],
                                value=0,disabled=False,continuous_update=False,readout=True)
wsaveboolaaa =widgets.ToggleButton(value=False,description='Savefig',disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',icon='check')

g = interactive(plotRaDec2D, infilename=filenames,field = columns, nside=wnside, title='', fsize=18, 
                outfilename='', cmap_name=cmaps, vmax=vmax,contrast=contrast, smooth=False,fwhm=wfwhm, 
                savefig=wsaveboolaaa )



In [ ]:
display(g)

In [ ]:
#Find closest number of pixel per object
    #nobj = len(data); #print(nobj)
    #npix_approx=12*(int(np.sqrt(nobj//12))**2); 
    #nside = hp.npix2nside(npix_approx); #print(nside)
    #i=1; 
    #while(nside>0): nside=nside//2; i+=1;
    #nside =2**i; #print(nside)
    #npixels = hp.nside2npix(nside); #print(npixels)